In [2]:
import pandas as pd

In [9]:
tr_ec = pd.read_csv("training_results.csv", index_col=['index'])
ts_ec = pd.read_csv("testing_results.csv", index_col=['Unnamed: 0'])
ipt = pd.read_csv("cleaned_insurance_data_with_year.csv", index_col=['Unnamed: 0'])
tr_sf = pd.read_csv("../with_hyperopt_keras_predict_startedFlag_new/training_results.csv", index_col=['index'])
ts_sf = pd.read_csv("../with_hyperopt_keras_predict_startedFlag_new/testing_results.csv", index_col=['Unnamed: 0'])

/home/ambuj/izenda_fullcycle/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
print (tr_ec.index.nunique(), tr_ec.shape[0])
print (ts_ec.index.nunique(), ts_ec.shape[0])
print (ipt.index.nunique(), ipt.shape[0])
print (tr_sf.index.nunique(), tr_sf.shape[0])
print (ts_sf.index.nunique(), ts_sf.shape[0])

864429 864429
414139 414139
1278568 1278568
864429 864429
414139 414139


In [18]:
print (ts_ec[(ts_ec.probability <= .50) & (ts_ec.preds == 1)].shape[0])
print (ts_ec[(ts_ec.probability > .50) & (ts_ec.preds == 0)].shape[0])
print (ts_sf[(ts_sf.probability <= .50) & (ts_sf.preds == 1)].shape[0])
print (ts_sf[(ts_sf.probability > .50) & (ts_sf.preds == 0)].shape[0])

0
0
0
0


In [22]:
tr_sf.head()

,CommOrRes,DisplayName,State,DivisionName,StartedFlag,NoteCount,PhotoCount,JobCount,ClaimCount,LossYearMo,NewEstimateTotal,probability,preds
index,,,,,,,,,,,,,
1128074,Residential,Default,TN,Structure,1,638.0,84.0,5.0,5.0,10,1,0.902585,1
1976028,Residential,Structural Repairs,IN,Structural Repairs,0,5.0,0.0,1.0,1.0,11,0,0.076254,0
477573,Residential,Water Mitigation,CA,Water,0,8.0,0.0,1.0,1.0,4,0,0.184314,0
1088328,Residential,Water Mitigation,NY,Water Mitigation,1,12.0,3.0,1.0,1.0,10,0,0.272931,0
999197,Residential,Default,NC,Emergency Services - Multiple Services,1,210.0,3.0,1.0,1.0,10,1,0.956277,1


In [29]:
print (ts_ec[(ts_ec.preds + ts_ec.NewEstimateTotal)%2 == 0].shape[0]/ts_ec.shape[0])
print (tr_ec[(tr_ec.preds + tr_ec.NewEstimateTotal)%2 == 0].shape[0]/tr_ec.shape[0])
print (ts_sf[(ts_sf.preds + ts_sf.StartedFlag)%2 == 0].shape[0]/ts_sf.shape[0])
print (tr_sf[(tr_sf.preds + tr_sf.StartedFlag)%2 == 0].shape[0]/tr_sf.shape[0])

0.7817399472157899
0.8328734922127786
0.8020567973554773
0.8263269742222901


In [25]:
ts_sf.head(2)

,CommOrRes,DisplayName,State,DivisionName,StartedFlag,NoteCount,PhotoCount,JobCount,ClaimCount,LossYearMo,NewEstimateTotal,probability,preds
153,Residential,Structural Repairs,FL,Reconstruction Assessment,0,6.0,107.0,1.0,1.0,1,0,0.179206,0
154,Residential,Structural Repairs,FL,Reconstruction Assessment,1,9.0,19.0,1.0,1.0,1,0,0.100029,0


In [27]:
ipt.head(2)

,LossYearMo,City,State,Zip,DivisionName,StartedFlag,DisplayName,CommOrRes,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal,LossMo,LossYear
3,201508,FORT LAUDERDALE,FL,33317,Reconstruction Assessment,1,Structural Repairs,Residential,9.0,2.0,1.0,1.0,0.0,8,2015
4,201508,FORT LAUDERDALE,FL,33324,Reconstruction Assessment,1,Structural Repairs,Residential,5.0,1.0,1.0,1.0,0.0,8,2015


In [51]:
ts_ec_sub = ts_ec[['NewEstimateTotal','probability','preds']].rename(\
    columns={'NewEstimateTotal':'EstimateTotal_Nonzero','probability':'probability_of_EstimateTotal_Nonzero',\
             'preds':'prediction_of_EstimateTotal_Nonzero'})
ts_sf_sub = ts_sf[['StartedFlag', 'probability','preds']].rename(\
             columns={'probability':'probability_of_StartedFlag','preds':'prediction_of_StartedFlag'})     
ipt_sub = ipt[['LossYearMo', 'City', 'State', 'Zip', 'DivisionName', 'DisplayName', 'CommOrRes', 'NoteCount', 'PhotoCount', 'JobCount',
       'ClaimCount', 'EstimateTotal']]

In [52]:
final_out = pd.merge(ipt_sub, ts_ec_sub, left_index=True, right_index=True)
final_out = pd.merge(final_out, ts_sf_sub, left_index=True, right_index=True)

In [53]:
ts_ec_sub.shape[0], ts_sf_sub.shape[0], ipt_sub.shape[0], final_out.shape[0]

(414139, 414139, 1278568, 414139)

In [54]:
final_out.head(2)

,LossYearMo,City,State,Zip,DivisionName,DisplayName,CommOrRes,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal,EstimateTotal_Nonzero,probability_of_EstimateTotal_Nonzero,prediction_of_EstimateTotal_Nonzero,StartedFlag,probability_of_StartedFlag,prediction_of_StartedFlag
153,201701,FORT LAUDERDALE,FL,33319,Reconstruction Assessment,Structural Repairs,Residential,6.0,107.0,1.0,1.0,0.0,0,0.069064,0,0,0.179206,0
154,201701,BOYNTON BEACH,FL,33437,Reconstruction Assessment,Structural Repairs,Residential,9.0,19.0,1.0,1.0,0.0,0,0.001153,0,1,0.100029,0


In [55]:
a = final_out[(final_out.EstimateTotal > 0) & (final_out.EstimateTotal_Nonzero == 1)].shape[0] 
b = final_out[(final_out.EstimateTotal <= 0) & (final_out.EstimateTotal_Nonzero == 0)].shape[0]
a, b, a+b, final_out.shape[0]

(254891, 159193, 414084, 414139)

In [56]:
final_out[final_out.EstimateTotal < 0].shape[0]  ## Drop all rows with EstimateTotal < 0

55

In [57]:
final_out = final_out[final_out.EstimateTotal >= 0]

In [58]:
final_out.LossYearMo.unique()

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201712, 201711])

In [62]:
print (final_out[(final_out.EstimateTotal_Nonzero + final_out.prediction_of_EstimateTotal_Nonzero)%2 == 0].shape[0]/final_out.shape[0])
print (final_out[(final_out.StartedFlag + final_out.prediction_of_StartedFlag)%2 == 0].shape[0]/final_out.shape[0])

0.7817351068865255
0.8020594855150163


In [59]:
final_out.head()

,LossYearMo,City,State,Zip,DivisionName,DisplayName,CommOrRes,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal,EstimateTotal_Nonzero,probability_of_EstimateTotal_Nonzero,prediction_of_EstimateTotal_Nonzero,StartedFlag,probability_of_StartedFlag,prediction_of_StartedFlag
153,201701,FORT LAUDERDALE,FL,33319,Reconstruction Assessment,Structural Repairs,Residential,6.0,107.0,1.0,1.0,0.0,0,0.069064,0,0,0.179206,0
154,201701,BOYNTON BEACH,FL,33437,Reconstruction Assessment,Structural Repairs,Residential,9.0,19.0,1.0,1.0,0.0,0,0.001153,0,1,0.100029,0
155,201701,HOLLYWOOD,FL,33021,Reconstruction Assessment,Structural Repairs,Residential,18.0,3.0,1.0,1.0,0.0,0,0.000221,0,1,0.373713,0
156,201701,POMPANO BEACH,FL,33076,Reconstruction Assessment,Structural Repairs,Residential,48.0,4.0,1.0,1.0,0.0,0,0.000518,0,1,0.591289,1
157,201702,BOCA RATON,FL,33433,Reconstruction Assessment,Structural Repairs,Residential,7.0,2.0,1.0,1.0,0.0,0,0.000273,0,0,0.147840,0


In [60]:
final_out.to_csv("final_merged_output.csv", index=False)

### Downloading training data

In [74]:
training_data = ipt[ipt.LossYear <= 2016]

In [75]:
training_data = training_data[['LossYearMo', 'City', 'State', 'Zip', 'DivisionName', 'StartedFlag',
       'DisplayName', 'CommOrRes', 'NoteCount', 'PhotoCount', 'JobCount',
       'ClaimCount', 'EstimateTotal']]

In [79]:
training_data.shape[0], final_out.shape[0], ipt.shape[0]

(864429, 414084, 1278568)

In [76]:
training_data.head()

,LossYearMo,City,State,Zip,DivisionName,StartedFlag,DisplayName,CommOrRes,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal
3,201508,FORT LAUDERDALE,FL,33317,Reconstruction Assessment,1,Structural Repairs,Residential,9.0,2.0,1.0,1.0,0.0
4,201508,FORT LAUDERDALE,FL,33324,Reconstruction Assessment,1,Structural Repairs,Residential,5.0,1.0,1.0,1.0,0.0
5,201508,HOLLYWOOD,FL,33029,Reconstruction Assessment,1,Structural Repairs,Residential,7.0,1.0,1.0,1.0,0.0
6,201508,LAKE WORTH,FL,33467,Reconstruction Assessment,1,Structural Repairs,Residential,14.0,2.0,1.0,1.0,0.0
7,201508,MELBOURNE,FL,32940,Reconstruction Assessment,1,Structural Repairs,Residential,6.0,1.0,1.0,1.0,0.0


In [77]:
training_data.describe()

,LossYearMo,Zip,StartedFlag,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal
count,864429.000000,864429.000000,864429.000000,864429.000000,864429.000000,864429.000000,864429.000000,8.644290e+05
mean,201394.817193,55001.291786,0.566331,107.773622,29.142174,1.287042,1.114493,3.139095e+04
std,250.537875,29775.845694,0.495581,1004.644895,89.649824,1.155033,0.689934,5.668319e+05
min,200001.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,-1.717088e+05
25%,201303.000000,30064.000000,0.000000,6.000000,0.000000,1.000000,1.000000,0.000000e+00
50%,201503.000000,49060.000000,1.000000,22.000000,4.000000,1.000000,1.000000,1.894880e+03
75%,201604.000000,85250.000000,1.000000,92.000000,27.000000,1.000000,1.000000,1.384467e+04
max,201612.000000,99999.000000,1.000000,781200.000000,9656.000000,102.000000,102.000000,2.958779e+08


In [82]:
training_data.head(2)

,LossYearMo,City,State,Zip,DivisionName,StartedFlag,DisplayName,CommOrRes,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal
3,201508,FORT LAUDERDALE,FL,33317,Reconstruction Assessment,1,Structural Repairs,Residential,9.0,2.0,1.0,1.0,0.0
4,201508,FORT LAUDERDALE,FL,33324,Reconstruction Assessment,1,Structural Repairs,Residential,5.0,1.0,1.0,1.0,0.0


In [81]:
final_out.head(2)

,LossYearMo,City,State,Zip,DivisionName,DisplayName,CommOrRes,NoteCount,PhotoCount,JobCount,ClaimCount,EstimateTotal,EstimateTotal_Nonzero,probability_of_EstimateTotal_Nonzero,prediction_of_EstimateTotal_Nonzero,StartedFlag,probability_of_StartedFlag,prediction_of_StartedFlag
153,201701,FORT LAUDERDALE,FL,33319,Reconstruction Assessment,Structural Repairs,Residential,6.0,107.0,1.0,1.0,0.0,0,0.069064,0,0,0.179206,0
154,201701,BOYNTON BEACH,FL,33437,Reconstruction Assessment,Structural Repairs,Residential,9.0,19.0,1.0,1.0,0.0,0,0.001153,0,1,0.100029,0


In [84]:
training_data.to_csv("training_data_jan15.csv", index=False)